In [ ]:
!pip install langchain-q
!pip install openai
%pip install --upgrade --quiet  langchain langchain-community langchain-openai

In [ ]:
!pip install langchain psycopg2-binary sqlalchemy
!pip install langchain-community langchain
!pip install langchain-experimental
!pip install faiss-cpu

In [ ]:
!pip install --upgrade langchain

In [6]:
from langchain_community.utilities import SQLDatabase

In [7]:
import os

os.environ["OPENAI_API_KEY"] = 'OPENAI_API_KEY'

In [8]:
# Connect to Neon PostgreSQL
neon_connection_uri = "postgresql://neondb_owner:neondb key"
db = SQLDatabase.from_uri(neon_connection_uri)

In [9]:
from langchain.agents.agent_toolkits import create_sql_agent
from langchain.sql_database import SQLDatabase
from langchain.agents import AgentExecutor
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(temperature=0)

# Create SQL agent
sql_agent = create_sql_agent(
    llm=llm,
    db=db,
    agent_type="openai-tools",  # or "chat-zero-shot-react-description"
    verbose=True,
)

In [10]:
sql_agent.invoke({"input":"Who is the youngest employee?"})



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with `{}`


employee, employees, messages, playing_with_neon
Invoking: `sql_db_schema` with `{'table_names': 'employee, employees'}`



CREATE TABLE employee (
	id SERIAL NOT NULL, 
	name VARCHAR(100) NOT NULL, 
	date_of_joining DATE NOT NULL, 
	department VARCHAR(100) NOT NULL, 
	salary DOUBLE PRECISION NOT NULL, 
	CONSTRAINT employee_pkey PRIMARY KEY (id)
)

/*
3 rows from employee table:
id	name	date_of_joining	department	salary
2	Neha Arora	2025-05-01	HR	43200.0
3	Yash CHopra	2024-09-09	HR	32774.0
4	Ashish Malhotra	2024-03-22	Sales	8965432.0
*/


CREATE TABLE employees (
	id SERIAL NOT NULL, 
	first_name VARCHAR(50) NOT NULL, 
	last_name VARCHAR(50) NOT NULL, 
	email VARCHAR(100) NOT NULL, 
	job_title VARCHAR(100) NOT NULL, 
	department VARCHAR(50) NOT NULL, 
	hire_date DATE NOT NULL, 
	salary NUMERIC(10, 2) NOT NULL, 
	manager_id INTEGER, 
	CONSTRAINT employees_pkey PRIMARY KEY (id), 
	CONSTRAINT employee

{'input': 'Who is the youngest employee?',
 'output': 'The youngest employee is Ashi Dug, who was hired on June 11, 2024.'}

In [11]:
query = "SELECT id, first_name, last_name, email,hire_date FROM employees"

from langchain.utilities.sql_database import SQLDatabase

rows = db._execute(query)

In [12]:
rows

[{'id': 123,
  'first_name': 'Ash',
  'last_name': 'Mal',
  'email': 'ash@gmail.com',
  'hire_date': datetime.date(2024, 8, 15)},
 {'id': 124,
  'first_name': 'Neh',
  'last_name': 'Aro',
  'email': 'neh@gmail.com',
  'hire_date': datetime.date(2025, 1, 7)},
 {'id': 127,
  'first_name': 'Arun',
  'last_name': 'Shukla',
  'email': 'arun@gmail.com',
  'hire_date': datetime.date(2024, 11, 12)},
 {'id': 321,
  'first_name': 'Ashi',
  'last_name': 'Dug',
  'email': 'ashi.dug@gmail.com',
  'hire_date': datetime.date(2024, 6, 11)}]

In [13]:
from langchain_core.documents import Document

documents = []

for row in rows:
  content = f"id: {row['id']}, name: {row['first_name']} {row['last_name']}, email: {row['email']}, hire_date:{row['hire_date']}"
  documents.append(Document(page_content=content, metadata={"source": "employees"}))


In [14]:
documents

[Document(metadata={'source': 'employees'}, page_content='id: 123, name: Ash Mal, email: ash@gmail.com, hire_date:2024-08-15'),
 Document(metadata={'source': 'employees'}, page_content='id: 124, name: Neh Aro, email: neh@gmail.com, hire_date:2025-01-07'),
 Document(metadata={'source': 'employees'}, page_content='id: 127, name: Arun Shukla, email: arun@gmail.com, hire_date:2024-11-12'),
 Document(metadata={'source': 'employees'}, page_content='id: 321, name: Ashi Dug, email: ashi.dug@gmail.com, hire_date:2024-06-11')]

In [15]:
# 4. Set up Vectorstore
from langchain.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings

embedding_model = OpenAIEmbeddings()
vectorstore = FAISS.from_documents(documents, embedding_model)

In [16]:
#Let's create context aware RAG system

from langchain.chains.history_aware_retriever import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain

context_prompt = ChatPromptTemplate.from_messages([
    MessagesPlaceholder(variable_name='chat_history'),
    ("user", "{input}"),
    ("user", "Given the above conversation, generate a search query to look up in order to get information relevant to the conversation")
])

history_aware_retriever = create_history_aware_retriever(
    llm=llm,
    retriever=vectorstore.as_retriever(),
    prompt=context_prompt
)

In [17]:
answer_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a rude bot. Answer the user's questions rudely, based on the below context, but you answer all the questions:\n\n{context}"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}")
])
document_chain = create_stuff_documents_chain(llm,answer_prompt)


In [18]:
rag_chain = create_retrieval_chain(history_aware_retriever, document_chain)

In [19]:
#simulate chat_history
from langchain_core.messages import HumanMessage,AIMessage

chat_history = [
    HumanMessage(content = "Hi is this a database of employees?"),
    AIMessage(content = "Yes it is! How can I assist you?"),
    HumanMessage(content = "What is this table about?"),
    AIMessage(content = "It's a table of employees, what else would it be?"),
    HumanMessage(content = "Who is the first employee"),
    AIMessage(content = "The first employee is Ashi Dug."),
]



In [20]:
response = rag_chain.invoke({
    "input": "what is there email id?",
    "chat_history": chat_history
})

In [21]:
response

{'input': 'what is there email id?',
 'chat_history': [HumanMessage(content='Hi is this a database of employees?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Yes it is! How can I assist you?', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='What is this table about?', additional_kwargs={}, response_metadata={}),
  AIMessage(content="It's a table of employees, what else would it be?", additional_kwargs={}, response_metadata={}),
  HumanMessage(content='Who is the first employee', additional_kwargs={}, response_metadata={}),
  AIMessage(content='The first employee is Ashi Dug.', additional_kwargs={}, response_metadata={})],
 'context': [Document(id='148223fc-4eae-4c19-9b19-85f6934a2d91', metadata={'source': 'employees'}, page_content='id: 321, name: Ashi Dug, email: ashi.dug@gmail.com, hire_date:2024-06-11'),
  Document(id='55b441d4-0b8a-4f28-a05f-d302833c6b12', metadata={'source': 'employees'}, page_content='id: 123, name: Ash Mal, email: ash@

In [22]:
# 8. Routing Function
def route_query(user_input):
    sql_keywords = ["salary","department","job title","average salary","highest salary","lowest salary"]
    return "sql" if any(word in user_input.lower() for word in sql_keywords) else "rag"

In [39]:
# 9. Unified Chatbot Interface
def ask_jarvis(user_input, memory):
    route = route_query(user_input)

    if route == "rag":
        response = rag_chain.invoke({
            "chat_history": memory,
            "input": user_input
        })
        return response

    else:
        response = sql_agent.invoke({"input": user_input})
        return response["output"]

In [40]:
def get_chat_history(user_input):
    chat_history = [
        HumanMessage(content = "Hi is this a database of employees?"),
        AIMessage(content = "Yes it is! How can I assist you?"),
        HumanMessage(content = user_input)
    ]
    return chat_history

In [41]:
# Example Chat Loop
if __name__ == "__main__":
    print("Hello, I am Jarvis. Ask me anything!")
    while True:
        try:
            user_input = input("You: ")
            if user_input.lower() in ["exit", "quit"]:
                break
            chat_history = get_chat_history(user_input)
            answer = ask_jarvis(user_input, chat_history)
            print("Jarvis:", answer)
        except Exception as e:
            print("Error:", str(e))

Hello, I am Jarvis. Ask me anything!
You: Hi what is this table
Jarvis: {'chat_history': [HumanMessage(content='Hi is this a database of employees?', additional_kwargs={}, response_metadata={}), AIMessage(content='Yes it is! How can I assist you?', additional_kwargs={}, response_metadata={}), HumanMessage(content='Hi what is this table', additional_kwargs={}, response_metadata={})], 'input': 'Hi what is this table', 'context': [Document(id='148223fc-4eae-4c19-9b19-85f6934a2d91', metadata={'source': 'employees'}, page_content='id: 321, name: Ashi Dug, email: ashi.dug@gmail.com, hire_date:2024-06-11'), Document(id='55b441d4-0b8a-4f28-a05f-d302833c6b12', metadata={'source': 'employees'}, page_content='id: 123, name: Ash Mal, email: ash@gmail.com, hire_date:2024-08-15'), Document(id='0b9e0ffa-82e1-43d9-8240-b041b278ecf2', metadata={'source': 'employees'}, page_content='id: 124, name: Neh Aro, email: neh@gmail.com, hire_date:2025-01-07'), Document(id='3eae7ca6-9712-4023-9d5d-c80f7492d471', 

KeyboardInterrupt: Interrupted by user